In [1]:
import cv2
import csv
from cv2 import destroyAllWindows
import mediapipe as mp
import numpy as np
import pandas as pd
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose


In [2]:
def calculate_angle(a, b, c):
    a = np.array(a)  
    b = np.array(b)  
    c = np.array(c)  
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - \
        np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    if angle > 180.0:
        angle = 360-angle
    return angle


In [3]:
import math
def get_distance(x,y):
    distance = math.sqrt((x[0]-y[0])**2 + (x[1]-y[1])**2)
    return distance * 100

    

In [4]:
def rescale_frame(frame, percent=50):
    width = int(frame.shape[1] * percent / 100)
    height = int(frame.shape[0] * percent / 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation=cv2.INTER_AREA)


In [13]:


cap = cv2.VideoCapture("hel.mp4")
stage = "Analyzing"


width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) + 0.5)
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) + 0.5)
size = (640, 480)
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter('output_video_.mp4', fourcc, 24, size)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if frame is not None:
            frame_ = rescale_frame(frame, percent=150)

        # Recolor image to RGB
        image = cv2.cvtColor(frame_, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark

            # Get coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                   landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                    landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            foot_index = [landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y]

            shoulder_r = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            elbow_r = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            wrist_r = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                    landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            hip_r = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            knee_r = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                    landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            ankle_r = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                    landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            foot_index_r = [landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x,
                    landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y]
            # Calculate angle
            elbow = calculate_angle(shoulder, elbow, wrist)
            elbow_r = calculate_angle(shoulder_r, elbow_r, wrist_r)
            angle_knee = calculate_angle(hip, knee, ankle)  
            angle_knee = round(angle_knee, 2)
            angle_knee_r = calculate_angle(hip_r, knee_r, ankle_r)
            angle_knee_r = round(angle_knee_r, 2)
            angle_hip = calculate_angle(shoulder, hip, knee)
            angle_hip = round(angle_hip, 2)
            angle_hip_r = calculate_angle(shoulder_r, hip_r, knee_r)
            angle_hip_r = round(angle_hip_r, 2)
            angle_ankle = calculate_angle(knee, ankle, foot_index)
            angle_ankle = round(angle_ankle, 2)
            angle_ankle_r = calculate_angle(knee_r, ankle_r, foot_index_r)
            angle_ankle_r = round(angle_ankle_r, 2)
            elbow = round(elbow, 2)
            elbow_r = round(elbow_r, 2)
            hip_angle = 180-angle_hip
            knee_angle = 180-angle_knee
            ankle_angle = 180-angle_ankle
            hip_angle_r = 180-angle_hip_r
            knee_angle_r = 180-angle_knee_r
            ankle_angle_r = 180-angle_ankle_r

            # step length
            step_length = get_distance(ankle, ankle_r)
            step_length = round(step_length, 2)
            # csv
            with open('angle_dc.csv', 'a+') as csvFile:
                # store float values
                writer = csv.writer(csvFile)
                writer.writerow([angle_knee, angle_knee_r, angle_hip, angle_hip_r, angle_ankle, angle_ankle_r,elbow, elbow_r, step_length])
        except:
            pass

        # Setup status box
        cv2.rectangle(image, (20,20), (200,90), (0,0,0), -1)

        cv2.rectangle(image, (20, 20), (200, 100), (255, 255, 255), -1)

        # Hip angle:
        cv2.putText(image, "Hip-joint angle : " + str(angle_hip),
                    (30, 95),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1, cv2.LINE_AA)

        # Knee angle:
        cv2.putText(image, "Knee-joint angle : " + str(angle_knee),
                    (30, 75),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1, cv2.LINE_AA)

        # ankle angle:
        cv2.putText(image, "ankle-joint angle : " + str(angle_ankle),
                    (30, 55),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1, cv2.LINE_AA)

        cv2.putText(image, "Stage : " + str(stage),
                    (30, 35),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1, cv2.LINE_AA)

        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(
                                      color=(0, 0, 255), thickness=2, circle_radius=1),
                                  mp_drawing.DrawingSpec(
                                      color=(0, 255, 0), thickness=2, circle_radius=1)
                                  )

        out.write(image)
        cv2.imshow('Pose & angle Frame', image)
        # destroy all windows in 22 seconds
        if cv2.waitKey(22) & 0xFF == ord('q'):
            cap.release()
            out.release()
            cv2.destroyAllWindows()
            break
    cap.release()
    out.release()
    cv2.destroyAllWindows()


In [15]:
load_csv = pd.read_csv('angle_dc.csv')

In [16]:
# show the data head
load_csv.head()

,knee,knee_r,hip,hip_r,ankle,ankle_r,elbow,elbow_r,step_length
0,169.20,144.44,176.60,178.48,122.60,144.44,174.33,179.88,11.30
1,171.56,143.82,177.94,178.62,124.54,143.82,174.94,178.92,11.04
2,171.12,143.22,177.69,179.47,124.79,143.22,174.86,176.64,10.50
3,169.13,143.04,176.96,179.45,125.50,143.04,176.36,174.65,9.82
4,167.23,144.96,176.16,178.98,124.70,144.96,175.62,175.55,9.03


In [17]:
# find mean of all columns
load_csv.mean()


knee           162.162290
knee_r         166.722995
hip            171.469950
hip_r          173.449121
ankle          132.296535
ankle_r        166.722995
elbow          170.777970
elbow_r        173.909876
step_length     12.092710
dtype: float64

In [18]:
with open('angle_dc_mean.csv', 'a+') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerow(load_csv.mean())

In [19]:
load_csv_mean = pd.read_csv('angle_dc_mean.csv')
load_csv_mean.head(5)

,angle_knee,angle_knee_r,angle_hip,angle_hip_r,angle_ankle,angle_ankle_r,elbow,elbow_r,step_length
0,162.554935,166.287500,171.503848,173.543478,162.554935,166.287500,170.468326,174.006413,11.916739
1,162.928596,165.393421,169.354035,170.144825,162.928596,165.393421,158.392193,154.828158,11.891842
2,166.545068,168.305534,171.518411,172.868822,166.545068,168.305534,163.652301,162.642521,8.689562
3,164.648714,166.411905,172.641667,173.930714,164.648714,166.411905,161.572095,164.306000,11.553000
4,159.267927,166.727683,171.232439,171.165854,159.267927,166.727683,167.983902,172.310732,10.525000


In [22]:
# save mean in new csv file
with open('angle_dc_mean_mean.csv', 'a+') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerow(load_csv_mean.mean())

In [26]:
load_csv_mean_mean = pd.read_csv('angle_dc_mean_mean.csv')
load_csv_mean_mean.head(5)

,knee,knee_r,hip,hip_r,ankle,ankle_r,elbow,elbow_r,step_length
0,164.051415,165.567538,170.696555,170.538656,163.477074,123.567182,157.517418,158.416282,10.601325


In [20]:
# clear csv file data without removing heading
with open('angle_dc.csv', 'w') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerow(['knee', 'knee_r', 'hip', 'hip_r', 'ankle', 'ankle_r', 'elbow', 'elbow_r', 'step_length'])